<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/group_svm/Statistical_tests_Group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Two Sample T-test Between Young Adult and Adolescent using Whole Brain voxels determined to be significant by the SVM. 

- Whole Brain Analyses
- Target Medial Prefrontal Cortex using Whole Brain SVM data to run t-test 
- Target Nucleus Accumbens using Whole Brain SVM data to run t-test 


In [ ]:
from google.colab import drive, files
drive.mount('/content/gdrive')#, force_remount = True)

# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/group_svm
!ls

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Import libraries
!pip install boto3 nilearn
import pickle
from visualize import *
from train import run_grp_svm_model
from analysis import create_bmaps, get_threshold_image, metrics, get_roi_bmaps
from process import *
from access_data import *

from sklearn.svm import SVC
import numpy as np
from collections import defaultdict
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt
from scipy import stats

> Load Young Adult Whole Brain Information

In [5]:
# Load model
#open path dictionary file to get subject ids
dict_path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(dict_path)

# Load in affine matrix - helps map the data coordinates to real world coordinates
affine_image = access_load_data('w3rtprun_01.nii',False)
t1_image = access_load_data('MNI152.nii',False)

mask_type = 'mask' #'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'
data_type = "YA_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
metric_ya = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
print(metric_ya.keys())

dict_keys(['bmap3', 'bmap2_3', 'alphas1', 'bmap', 'y_train', 'test_preds', 'test_probs', 'test_acc', 'test_dfunc', 'y_t'])


> Load Adolescent Whole Brain Information

In [6]:
# Load model

mask_type = 'mask' #'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'
data_type = "AD_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
metric_ad = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
print(metric_ad.keys())

dict_keys(['bmap3', 'bmap2_3', 'alphas1', 'bmap', 'y_train', 'test_preds', 'test_probs', 'test_acc', 'test_dfunc', 'y_t'])


### Whole Brain Analysis Between Adolescent and Young Adult 2 Sample t-test

In [30]:
Young_adult = metric_ya['bmap'][0][0]
Adolescent = metric_ad['bmap'][0][0]

t_value, p_value = stats.ttest_ind(Young_adult, Adolescent)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')


Test statistic is 62.00711651640719
p-value for two tailed test is 0.0
Conclusion Since p-value 0.0 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


> Medial Prefrontal Cortex Analysis Between Adolescent and Young Adult 2 Sample t-test

In [32]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'mPFC' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'mPFC' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [34]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')


Test statistic is -4.131700751771177
p-value for two tailed test is 3.6011367639982006e-05
Conclusion Since p-value 3.6011367639982006e-05 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


> Nucleus Accumbens Analysis Between Adolescent and Young Adult 2 Sample t-test

In [35]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'nacc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ya = metric_ya['bmap2_3'][0]
bmap2ya = previous_bmap_ya.reshape(79 * 95 * 79)
bmap2_3ya = np.zeros((79, 95, 79))
bmap2_3ya = bmap2_3ya.reshape(79 * 95 * 79)
bmap2_3ya[indices_mask_roi] = bmap2ya[indices_mask_roi]

bmap2_3ya

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [36]:
# Load in the mask indices for mask overlay to create bmaps of a specific Region of interest
roi_path_ind = 1 # get mask data for region of interest
mask_roi = 'nacc_aal' #'acc_aal', 'anterior_insula_aal', 'mPFC', 'nacc_aal', 
indices_mask_roi = load_mask_indices(data_path_dict, mask_roi, roi_path_ind)

previous_bmap_ad = metric_ad['bmap2_3'][0]
bmap2ad = previous_bmap_ad.reshape(79 * 95 * 79)
bmap2_3ad = np.zeros((79, 95, 79))
bmap2_3ad = bmap2_3ad.reshape(79 * 95 * 79)
bmap2_3ad[indices_mask_roi] = bmap2ad[indices_mask_roi]

bmap2_3ad

mask shape: (79, 95, 79)


array([0., 0., 0., ..., 0., 0., 0.])

In [41]:
t_value, p_value = stats.ttest_ind(bmap2_3ya, bmap2_3ad)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value  {p_value}  < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion: Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.\
  \n and therefore the nucleus accumbens may play a role in regulation for both Young Adults and \
  \n Adolescents')

Test statistic is -1.449044263538568
p-value for two tailed test is 0.14732548612940824
Conclusion: Since p-value0.14732548612940824 > alpha 0.05,We do not reject the null hypothesis H0.  
 and therefore the nucleus accumbens may play a role in regulation for both Young Adults and   
 Adolescents
